In [1]:
# package(s) related to time, space and id
import datetime

# Used for mathematical functions
import math
import platform
import time

import matplotlib.pyplot as plt

# Used for making the graph to visualize our problem
import networkx as nx

# package(s) for data handling
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import itertools
import tqdm

import plotly.express as px
from plotly.subplots import make_subplots
# spatial libraries
import pyproj
import shapely.geometry

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy
from simplekml import Kml, Style



In [2]:
# V=ship velocity(VOG) or deep water velocity no current velocity was taking into account
# Urel=V+Ur
# Urel= 1/1-exp(4*(1-h/T))
# Am=midship cross section
# Ac=channel cross section
# T=ship draft

In [3]:
V=3
Bc=np.any([100])
h = np.array([2, 2.5, 3, 3.5 ,4, 5.5, 7, 10])
T = np.any([2.75])
r=h/T
print(r)

#Classification of water
for i in range(len(r)):
    if r[i] > 3:
        print(r[i], "not effect on ship")
    elif r[i] <= 3 and r[i]>1.5:
        print(r[i], "visible effect on ship")
    elif r[i] <=1.5 and r[i]>1.2:
        print(r[i], "very significant effect on ships")
    else:
        print(r[i], "water motion dominates ship") 
        
# Blockage coefficient Am/Ac
Bs=11.45
Ls=135
S=Bs*Ls+2*Ls*T
print('S=',S,'m^2')
Am=Bs*T
Ac=Bc*h
Blockage=Am/Ac
for i in range(len(Blockage)):
    if Blockage[i] <= 0.1:
        print("very wide channel Urel=Veff")
    else:
        print("urel=V+Ur") 
    
# Froude number
def get_Frh ():
        return V / np.sqrt(h*9.81)   
Frh = get_Frh () 
print('Frh', Frh)

# Frh<1 subcritical dn > dcritical
# Frh>1 supercritical dn < dcritical
def subcritical_or_supercritical (Frh):
    """Check flow"""
    if  np.all(Frh)<1:
        return "subcritical" 
    if  np.all(Frh)<0.65: 
        return"desirable condition"
    if  np.all(Frh)>1: # supercritical flow 
        return "unstable"
    print(subcritical_or_supercritical)
    return
Flow_type = subcritical_or_supercritical (Frh)
print (Flow_type)

# Return flow velocity
def get_Ur ():  
    return (Blockage * V)/(1-(Blockage)-(Frh**2))
Ur = get_Ur ()
print('Ur=', Ur, 'm/s')

# Effective velocity
def get_Veff (): 
    return V/(1-np.exp(4*(1- r)))
Veff = get_Veff ()
print('Veff=', Veff, 'm/s') 

# Relative velocity
def get_Urel ():
    if np.all(Blockage) <=0.1:
        Urel= Veff
    else:
        Urel=  V + Ur
    return Urel
Urel = get_Urel ()
print('Urel=', Urel, 'm/s')

[ 2.   2.5  3.   3.5  4.   5.5  7.  10. ]
2.0 visible effect on ship
2.5 visible effect on ship
3.0 visible effect on ship
3.5 not effect on ship
4.0 not effect on ship
5.5 not effect on ship
7.0 not effect on ship
10.0 not effect on ship
S= 1815.75 m^2
urel=V+Ur
urel=V+Ur
urel=V+Ur
urel=V+Ur
urel=V+Ur
urel=V+Ur
urel=V+Ur
urel=V+Ur
Frh [0.67728546 0.60578253 0.55300126 0.51197969 0.47891314 0.4084185
 0.36202431 0.30289127]
<function subcritical_or_supercritical at 0x0000023BE8C02EE0>
None
Ur= [ -3.31326048  -3.48114918  -3.66696048  -3.87372614  -4.10520257
  -5.00187028  -6.39970943 -14.5093974 ] m/s
Veff= [3.05597208 3.00745473 3.00100673 3.00013621 3.00001843 3.00000005
 3.         3.        ] m/s
Urel= [ -0.31326048  -0.48114918  -0.66696048  -0.87372614  -1.10520257
  -2.00187028  -3.39970943 -11.5093974 ] m/s


## U_rel calculation and plotting starts from here 
- define function
- set input value
- arrange input table
- use the defined function and get output table
- select desired variables for plotting

#### define function

In [4]:
# Adjusted velocity 
def get_Vadj (Bs, Bc, V, T, h, Ls):
   
    # get blockage factor
    Am = Bs * T
    Ac = Bc * h
    Blockage = Am / Ac
    r = h/T
    
    #get limit speed 1
    Vlim1 = np.sqrt (9.81*Ls/2*np.pi)
    
    #get limit speed 
    Vlim2 = 0.78 * (np.sqrt(h*9.81)) * ((1 - Blockage)**2.25)
    
    # get shallow speed     
    r = h/T
    Vsh = 1.18 * V * ((1/1-np.exp(4*(1-max([r, 1.1]))))**(-0.75 - Blockage))
    
    #get adjusted velocity
    if Vsh <= 0.9 * Vlim2:
        Vadj = Vsh
    if V > 0.9 * Vlim2 and V > Vsh:
        Vadj = V 
    if Vsh > V > 0.9 * Vlim2 or 0.9 * Vlim2 > V > Vsh:
        Vadj = 0.9 * Vlim2 
    else:
        Vadj = V
    return r, Vadj

# Return current speed 
def get_Ur (Bs, Bc, Vadj, T, h):
    Am = Bs * T
    Ac = Bc * h
    Blockage = Am / Ac
    
    # get Froude number
    Frh = Vadj / np.sqrt(h * 9.81)
    Ur = (Blockage * Vadj)/(1-(Blockage)-(Frh**2))
    
    return Ur

# Relative velocity
def get_Urel (h, T, Ur, Vadj): 
    Unarrow = Ur + Vadj
    Ushallow = (1/1-np.exp(4*(1-max([h/T, 1.1])))) * Vadj
    Urel = max([Unarrow, Ushallow])
    
    return Urel, Unarrow, Ushallow


####Rf
def get_Rf (Bs, Urel, T, Ls):
    
    # get cf
    Re = (Urel*Ls)/ (10**(-6))
    cf = 0.053 /((np.log10(Re)-2)**2)
    
    Rf = 0.5 *(Urel**2) * cf * (Bs *Ls + 2 * Ls * T)
    return  Rf

####Rp
def get_Rp (Vadj, T):
    Rp = 0.5 *(Vadj**2) * 0.15 * (Bs * T)
    return Rp

####Rz
def get_Rz (Bs, Bc, T, h, Vadj):
    
    # get blockage factor
    Am = Bs * T
    Ac = Bc * h
    Blockage = Am / Ac
    
    # get Froude number
    Frh = Vadj / np.sqrt(h*9.81)
    
    #get z
    z = (Blockage/(1-Blockage-(Frh**2))) * (Frh**2) * h
    
    Rz = 9.81 *z * 0.2 * (Bs * T)
    return Rz

####Rtot
def get_Rtot(Rp, Rf, Rz):
    
    Rtot =  Rp + Rf + Rz
    return Rtot

# Power estimation 
# BHP > DHP > EHP
# 1) Effective horse power Pe (Power needed to overcome the total drag)
def get_Pe ():
    Pe = Urel * Rtot
    return Pe

# 2) Delivered horse power Pd (Power delivered to the propeller shaft)
def get_Pd (Pe):
    Pd = Pe/ 0.55
    return Pd

# 3) Brake horse power or Propulsion power Pb (Required motor power)
def get_Pb (Pe):
    Pb = 2*Pe
    return Pb

#### input value for Urel

In [5]:
h = np.linspace(3.03,10,71)  # water depth from 3 m to 10 m is a suitable range for the Ductch waterways 
h_more = np.linspace(3.03,10,29)
V_more = np.linspace(0,5,21)
Bs = [9.5]
Bc= [50] 
V = [4]
T = [3]
Ls = [100]

#### prepare input matrix for calculation

In [6]:
# prepare the work to be done
# create a list of all combinations
work = list(itertools.product( h_more, Bs, Bc, V_more, T, Ls))

# prepare a list of dictionaries for pandas
rows = []
for item in work:
    row = {"h_more": item[0],"Bs": item[1], "Bc": item[2], "V_more": item[3], "T": item[4], "Ls": item[5]}
    rows.append(row)

# these are all the simulations that we want to run
# convert them to dataframe, so that we can apply a function and monitor progress
work_df = pd.DataFrame(rows)
work_df.head(500)

,h_more,Bs,Bc,V_more,T,Ls
0,3.030000,9.5,50,0.00,3,100
1,3.030000,9.5,50,0.25,3,100
2,3.030000,9.5,50,0.50,3,100
3,3.030000,9.5,50,0.75,3,100
4,3.030000,9.5,50,1.00,3,100
...,...,...,...,...,...,...
495,8.755357,9.5,50,3.00,3,100
496,8.755357,9.5,50,3.25,3,100
497,8.755357,9.5,50,3.50,3,100
498,8.755357,9.5,50,3.75,3,100


#### use the defined function and get output table

In [7]:

results = []

for i, row in tqdm.tqdm(work_df.iterrows()):
    # create a new vessel, like the one above (so that it also has L)
    h = row['h_more']
    Bs = row['Bs']
    Bc = row['Bc']
    V = row['V_more']
    T = row['T']
    Ls = row['Ls']

    r, Vadj = get_Vadj(Bs, Bc, V, T, h, Ls)
    Ur = get_Ur(Bs, Bc, Vadj, T, h)
    Urel, Unarrow, Ushallow = get_Urel (h, T, Ur, Vadj)
    Rf = get_Rf(Bs, Urel, T, Ls)
    Rp = get_Rp(Vadj, T)
    Rz = get_Rz (Bs, Bc, T, h, Vadj)
    Rtot = get_Rtot(Rp, Rf, Rz)
    Pe = get_Pe ()
    Pb = get_Pb (Pe)
    Pd = get_Pd (Pe)


        
    result = {}
    result.update(row)
    result['h/T'] = r
    result['Vadj'] = Vadj
    result['Ur'] = Ur
    result['Urel'] = Urel
    result['Unarrow'] = Unarrow
    result['Ushallow'] = Ushallow
    result['Rf'] = Rf
    result['Rp'] = Rp
    result['Rz'] = Rz
    result['Rtot']= Rtot
    result['Pe'] = Pe
    result['Pb'] = Pb
    result['Pd'] = Pd

    results.append(result)

plot_df = pd.DataFrame(results)

plot_df.head(100)

0it [00:00, ?it/s]C:\Users\30698\AppData\Local\Temp\ipykernel_149816\2004362169.py:55: RuntimeWarning: divide by zero encountered in log10
  cf = 0.053 /((np.log10(Re)-2)**2)
3it [00:00, ?it/s]


UnboundLocalError: local variable 'Vadj' referenced before assignment

#### select desired variables for plotting

In [ ]:
selected_v3 = plot_df.query('V_more == 4')
selected_v2 = plot_df.query('V_more == 2.5')
plt.plot(selected_v3['h/T'], selected_v3['Vadj'], 'darkgreen', linewidth = 3.0)
plt.plot(selected_v2['h/T'], selected_v2['Vadj'], 'lightgreen', linewidth = 3.0)
plt.xlabel('h/T')
plt.ylabel('Vadj (m/s)')

plt.grid()
plt.xlim(1, 3.5)
plt.ylim()
plt.legend(['V =3 m/s','V =2.5 m/s'])
plt.title('Adjusted velocity');

In [ ]:
selected_v3 = plot_df.query('V_more == 4')
selected_v2 = plot_df.query('V_more == 2.5')
plt.plot(selected_v3['h/T'], selected_v3['Ushallow'], 'darkgreen', linewidth = 3.0)
plt.plot(selected_v2['h/T'], selected_v2['Ushallow'], 'lightgreen', linewidth = 3.0)
plt.xlabel('h/T')
plt.ylabel('Ushallow (m/s)')

plt.grid()
plt.xlim(1, 3.5)
plt.ylim()
plt.legend(['V =3 m/s','V =2.5 m/s'])
plt.title('Shallow velocity');

In [ ]:
selected_v3 = plot_df.query('V_more == 4')
selected_v2 = plot_df.query('V_more == 2.5')
plt.plot(selected_v3['h/T'], selected_v3['Unarrow'], 'darkgreen', linewidth = 3.0)
plt.plot(selected_v2['h/T'], selected_v2['Unarrow'], 'lightgreen', linewidth = 3.0)
plt.xlabel('h/T')
plt.ylabel('Unarrow (m/s)')

plt.grid()
plt.xlim(1, 3.5)
plt.ylim()
plt.legend(['V =3 m/s','V =2.5 m/s'])
plt.title('Narrow velocity');

In [ ]:
selected_v3 = plot_df.query('V_more == 3')
selected_v2 = plot_df.query('V_more == 2.5')
plt.plot(selected_v3['h/T'], selected_v3['Urel'], 'darkgreen', linewidth = 3.0)
plt.plot(selected_v2['h/T'], selected_v2['Urel'], 'lightgreen', linewidth = 3.0)
plt.xlabel('h/T')
plt.ylabel('Urel (m/s)')

plt.grid()
plt.xlim(1, 3.5)
plt.legend(['V =3 m/s','V =2.5 m/s'])
plt.title('Relative velocity');

In [ ]:
selected = plot_df.query('V_more == 4')



plt.plot(selected['h_more'], selected['Rtot'], 'black', linewidth = 3.0)
plt.plot(selected['h_more'], selected['Rf'], 'red', linewidth = 3.0)
plt.plot(selected['h_more'], selected['Rp'], 'green', linewidth = 3.0)
plt.plot(selected['h_more'], selected['Rz'], 'orange', linewidth = 3.0)
plt.xlabel('h (m)')
plt.ylabel('Ri (kN)')
plt.legend(['Rtot','Rf','Rp','Rz',])
plt.grid()
plt.xlim(3, 6)
plt.ylim()
plt.title('Resistance components at V = 4 m/s');

In [ ]:
selected = plot_df.query('V_more == 4')


plt.plot(selected['h_more'], selected['Pe'], 'purple', linewidth = 2.0)
plt.plot(selected['h_more'], selected['Pd'], 'gray', linewidth = 2.0)
plt.plot(selected['h_more'], selected['Pb'], 'pink', linewidth = 2.0)

plt.xlabel('h (m)')
plt.ylabel('Power (kW)')
plt.legend(['Pe','Pd','Pb'])
plt.grid()
plt.xlim(3, 6)
plt.ylim(0, 2000)
plt.title('Power estimation at V = 4 m/s');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more <= 4')
selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 4.2')
selected_h_4 = plot_df.query('h_more == 4')
selected_h_5 = plot_df.query('h_more == 5')
selected_h_6 = plot_df.query('h_more == 6')


plt.figure(dpi=400)
plt.plot(selected_h_3['V_more'],selected_h_3['Rf'], 'navy', linewidth = 1.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Rf'], 'r', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Rf'], 'red', linewidth = 1.0)
plt.plot(selected_h_5['V_more'],selected_h_5['Rf'], 'red', linewidth = 2.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Rf'], 'darkred', linewidth = 1)

# plt.plot(plot_df['Urel'], plot_df['Rp'], 'green', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rz'], 'orange', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rtot'], 'black', linewidth = 3.0)
plt.xlabel('V (m/s)')
plt.ylabel('Rf (kN)')
plt.legend(['Rf~3m','Rf~3.5m','Rf~4m','Rf~5m','Rf~6m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim()
plt.title('Frictional Resistance');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more <= 4')
selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 4.2')
selected_h_4 = plot_df.query('h_more == 4')
selected_h_5 = plot_df.query('h_more == 5')
selected_h_6 = plot_df.query('h_more == 6')


plt.figure(dpi=400)
plt.plot(selected_h_3['V_more'],selected_h_3['Rp'], 'navy', linewidth = 1.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Rp'], 'r', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Rp'], 'green', linewidth = 1.0)
plt.plot(selected_h_5['V_more'],selected_h_5['Rp'], 'green', linewidth = 2.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Rp'], 'darkgreen', linewidth = 1)

# plt.plot(plot_df['Urel'], plot_df['Rp'], 'green', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rz'], 'orange', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rtot'], 'black', linewidth = 3.0)
plt.xlabel('V (m/s)')
plt.ylabel('Rp (kN)')
plt.legend(['Rp~3m','Rp~3.5m','Rp~4m','Rp~5m','Rp~6m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim()
plt.title('Pressure Resistance');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more <= 4')
selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 4.2')
selected_h_4 = plot_df.query('h_more == 4')
selected_h_5 = plot_df.query('h_more == 5')
selected_h_6 = plot_df.query('h_more == 6')


plt.figure(dpi=400)
plt.plot(selected_h_3['V_more'],selected_h_3['Rz'], 'navy', linewidth = 1.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Rz'], 'r', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Rz'], 'orange', linewidth = 1.0)
plt.plot(selected_h_5['V_more'],selected_h_5['Rz'], 'orange', linewidth = 2.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Rz'], 'darkorange', linewidth = 1)

# plt.plot(plot_df['Urel'], plot_df['Rp'], 'green', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rz'], 'orange', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rtot'], 'black', linewidth = 3.0)
plt.xlabel('V (m/s)')
plt.ylabel('Rz (kN)')
plt.legend(['Rz~3m','Rz~3.5m','Rz~4m','Rz~5m','Rz~6m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim()
plt.title('Rz');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more <= 4')
selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 4.2')
selected_h_4 = plot_df.query('h_more == 4')
selected_h_5 = plot_df.query('h_more == 5')
selected_h_6 = plot_df.query('h_more == 6')



plt.plot(selected_h_3['V_more'],selected_h_3['Rtot'], 'navy', linewidth = 1.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Rtot'], 'r', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Rtot'], 'darkred', linewidth = 1.0)
plt.plot(selected_h_5['V_more'],selected_h_5['Rtot'], 'black', linewidth = 2.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Rtot'], 'gray', linewidth = 1)

# plt.plot(plot_df['Urel'], plot_df['Rp'], 'green', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rz'], 'orange', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rtot'], 'black', linewidth = 3.0)
plt.xlabel('V (m/s)')
plt.ylabel('Rtot (kN)')
plt.legend(['Rtot~3m','Rtot~3.5m','Rtot~4m','Rtot~5m','Rtot~6m'])
plt.xlim(0, 5)
plt.ylim()
plt.grid()
plt.title('Total Resistance');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more <= 4')
selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 4.2')
selected_h_4 = plot_df.query('h_more == 4')
selected_h_5 = plot_df.query('h_more == 5')
selected_h_6 = plot_df.query('h_more == 6')



plt.plot(selected_h_3['V_more'],selected_h_3['Rtot'], 'navy', linewidth = 1.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Rtot'], 'r', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Rtot'], 'darkred', linewidth = 1.0)
plt.plot(selected_h_5['V_more'],selected_h_5['Rtot'], 'black', linewidth = 2.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Rtot'], 'gray', linewidth = 1)

# plt.plot(plot_df['Urel'], plot_df['Rp'], 'green', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rz'], 'orange', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rtot'], 'black', linewidth = 3.0)
plt.xlabel('V (m/s)')
plt.ylabel('Rtot (kN)')
plt.legend(['Rtot~3m','Rtot~3.5m','Rtot~4m','Rtot~5m','Rtot~6m'])
plt.xlim(0, 3)
plt.ylim()
plt.grid()
plt.title('Total Resistance');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more <= 4')



plt.plot(selected_h_3['V_more'],selected_h_3['Rtot'], 'black', linewidth = 3.0)
plt.plot(selected_h_3['V_more'],selected_h_3['Rf'], 'red', linewidth = 4.0)
plt.plot(selected_h_3['V_more'],selected_h_3['Rp'], 'green', linewidth = 3.0)
plt.plot(selected_h_3['V_more'],selected_h_3['Rz'], 'orange', linewidth = 2)

plt.xlabel('V (m/s)')
plt.ylabel('R (kN)')
plt.legend(['Rtot~3m','Rf~3m','Rp~3m','Rz~3m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim()
plt.title('Resistance components at 3 m');

In [ ]:

selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 4.2')

plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Rtot'], 'black', linewidth = 3.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Rf'], 'red', linewidth = 4.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Rp'], 'green', linewidth = 3.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Rz'], 'orange', linewidth = 2)

plt.xlabel('V (m/s)')
plt.ylabel('R (kN)')
plt.legend(['Rtot~3.5m','Rf~3.5m','Rp~3.5m','Rz~3.5m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim(0, 180)
plt.title('Resistance components at 3.5 m');

In [ ]:
selected_h_4 = plot_df.query('h_more == 4')



plt.plot(selected_h_4['V_more'],selected_h_4['Rtot'], 'black', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Rf'], 'red', linewidth = 4.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Rp'], 'green', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Rz'], 'orange', linewidth = 2)


plt.xlabel('V (m/s)')
plt.ylabel('R (kN)')
plt.legend(['Rtot~4m','Rf~4m','Rp~4m','Rz~4m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim(0, 180)
plt.title('Resistance components at 4 m');

In [ ]:
selected_h_6 = plot_df.query('h_more == 6')



plt.plot(selected_h_6['V_more'],selected_h_6['Rtot'], 'black', linewidth = 3.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Rf'], 'red', linewidth = 4.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Rp'], 'green', linewidth = 3.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Rz'], 'orange', linewidth = 2)


plt.xlabel('V (m/s)')
plt.ylabel('R (kN)')
plt.legend(['Rtot~6m','Rf~6m','Rp~6m','Rz~6m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim(0, 180)
plt.title('Resistance components at 6 m');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more < 4')


plt.figure(dpi=400)
plt.plot(selected_h_3['V_more'],selected_h_3['Pe'], 'pink', linewidth = 3.0)
plt.plot(selected_h_3['V_more'],selected_h_3['Pb'], 'purple', linewidth = 3.0)
plt.plot(selected_h_3['V_more'],selected_h_3['Pd'], 'gray', linewidth = 3.0)


plt.xlabel('V (m/s)')
plt.ylabel('Power (kW)')
plt.legend(['Pe~3m','Pb~3m','Pd~3m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim()
plt.title('Power at 3 m');

In [ ]:
selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 4.2')


plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Pe'], 'pink', linewidth = 3.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Pb'], 'purple', linewidth = 3.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Pd'], 'gray', linewidth = 3.0)


plt.xlabel('V (m/s)')
plt.ylabel('Power (kW)')
plt.legend(['Pe~3.5m','Pb~3.5m','Pd~3.5m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim()
plt.title('Power at 3.5 m');

In [ ]:
selected_h_4 = plot_df.query('h_more == 4')



plt.plot(selected_h_4['V_more'],selected_h_4['Pe'], 'pink', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Pb'], 'purple', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Pd'], 'gray', linewidth = 3.0)


plt.xlabel('V (m/s)')
plt.ylabel('Power (kW)')
plt.legend(['Pe~4m','Pb~4m','Pd~4m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim(0, 2000)
plt.title('Power at 4 m');

In [ ]:
selected_h_4 = plot_df.query('h_more == 6')



plt.plot(selected_h_6['V_more'],selected_h_6['Pe'], 'pink', linewidth = 3.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Pb'], 'purple', linewidth = 3.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Pd'], 'gray', linewidth = 3.0)


plt.xlabel('V (m/s)')
plt.ylabel('Power (kW)')
plt.legend(['Pe~6m','Pb~6m','Pd~6m'])
plt.grid()
plt.xlim(0, 5)
plt.ylim(0, 2000)
plt.title('Power at 6 m');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more <= 4')
selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 4.2')
selected_h_4 = plot_df.query('h_more == 4')
selected_h_5 = plot_df.query('h_more == 5')
selected_h_6 = plot_df.query('h_more == 6')



plt.plot(selected_h_3['V_more'],selected_h_3['Pb'], 'navy', linewidth = 1.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Pb'], 'red', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Pb'], 'pink', linewidth = 2.5, linestyle = 'dotted')
plt.plot(selected_h_5['V_more'],selected_h_5['Pb'], 'pink', linewidth = 2.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Pb'], 'pink', linewidth = 1, linestyle = 'dashed')

# plt.plot(plot_df['Urel'], plot_df['Rp'], 'green', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rz'], 'orange', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rtot'], 'black', linewidth = 3.0)
plt.xlabel('V (m/s)')
plt.ylabel('Pb (kW)')
plt.legend(['Pb~3m','Pe~3.5m','Pb~4m','Pb~5m','Pb~6m'])
plt.xlim(0, 5)
plt.ylim()
plt.grid()
plt.title('Pb');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more <= 4')
selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 4.2')
selected_h_4 = plot_df.query('h_more == 4')
selected_h_5 = plot_df.query('h_more == 5')
selected_h_6 = plot_df.query('h_more == 6')



plt.plot(selected_h_3['V_more'],selected_h_3['Pd'], 'navy', linewidth = 1.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Pd'], 'red', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Pd'], 'gray', linewidth = 2.5, linestyle = 'dotted')
plt.plot(selected_h_5['V_more'],selected_h_5['Pd'], 'gray', linewidth = 2.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Pd'], 'gray', linewidth = 1, linestyle = 'dashed')

# plt.plot(plot_df['Urel'], plot_df['Rp'], 'green', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rz'], 'orange', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rtot'], 'black', linewidth = 3.0)
plt.xlabel('V (m/s)')
plt.ylabel('Pd (kW)')
plt.legend(['Pd~3m','Pe~3.5m','Pd~4m','Pd~5m','Pd~6m'])
plt.xlim(0, 5)
plt.ylim(0, 2000)
plt.grid()
plt.title('Pd');

In [ ]:
selected_h_3 = plot_df.query('h_more == 3 & V_more <= 4')
selected_h_3_5 = plot_df.query('h_more == 3.5 & V_more <= 4.2')
selected_h_4 = plot_df.query('h_more == 4')
selected_h_5 = plot_df.query('h_more == 5')
selected_h_6 = plot_df.query('h_more == 6')



plt.plot(selected_h_3['V_more'],selected_h_3['Pe'], 'navy', linewidth = 1.0)
plt.plot(selected_h_3_5['V_more'],selected_h_3_5['Pe'], 'red', linewidth = 3.0)
plt.plot(selected_h_4['V_more'],selected_h_4['Pe'], 'purple', linewidth = 2.5, linestyle = 'dotted')
plt.plot(selected_h_5['V_more'],selected_h_5['Pe'], 'purple', linewidth = 2.0)
plt.plot(selected_h_6['V_more'],selected_h_6['Pe'], 'purple', linewidth = 1, linestyle = 'dashed')

# plt.plot(plot_df['Urel'], plot_df['Rp'], 'green', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rz'], 'orange', linewidth = 3.0)
# plt.plot(plot_df['Urel'], plot_df['Rtot'], 'black', linewidth = 3.0)
plt.xlabel('V (m/s)')
plt.ylabel('Pe (kW)')
plt.legend(['Pe~3m','Pe~3.5m','Pe~4m','Pe~5m','Pe~6m'])
plt.xlim(0, 5)
plt.ylim(0, 2000)
plt.grid()
plt.title('Pe');